In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import fpgrowth

In [9]:
data= pd.read_csv('data/dataset.csv',encoding= 'unicode_escape')
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [10]:
#CLeaning
data['Description'] = data['Description'].str.strip()
data.dropna(axis=0, subset=['InvoiceNo'], inplace=True) 
data['InvoiceNo'] = data['InvoiceNo'].astype('str')
data = data[~data['InvoiceNo'].str.contains('C')] 
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [11]:
#Separating transactions
basket = (data.groupby(['InvoiceNo', 'Description'])['Quantity']
.sum().unstack().reset_index().fillna(0)
.set_index('InvoiceNo'))
basket.head()

Description,*Boombox Ipod Classic,*USB Office Mirror Ball,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 HANGING EGGS HAND PAINTED,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,...,wrongly coded 20713,wrongly coded 23343,wrongly coded-23343,wrongly marked,wrongly marked 23343,wrongly marked carton 22804,wrongly marked. 23343 in box,wrongly sold (22719) barcode,wrongly sold as sets,wrongly sold sets
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536366,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
#converting to 1 and  0
def encoder(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket = basket.applymap(encoder)
basket.drop('POSTAGE', inplace=True, axis=1, errors='ignore') # removing column named 'Postage'
basket.head()

Description,*Boombox Ipod Classic,*USB Office Mirror Ball,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 HANGING EGGS HAND PAINTED,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,...,wrongly coded 20713,wrongly coded 23343,wrongly coded-23343,wrongly marked,wrongly marked 23343,wrongly marked carton 22804,wrongly marked. 23343 in box,wrongly sold (22719) barcode,wrongly sold as sets,wrongly sold sets
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536366,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536367,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536368,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536369,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
#Generatig frequent itemsets
itemsets = fpgrowth(basket, min_support=0.01, use_colnames=True)
itemsets

,support,itemsets
0,0.109656,(WHITE HANGING HEART T-LIGHT HOLDER)
1,0.022271,(KNITTED UNION FLAG HOT WATER BOTTLE)
2,0.020815,(RED WOOLLY HOTTIE WHITE HEART.)
3,0.018438,(SET 7 BABUSHKA NESTING BOXES)
4,0.014896,(WHITE METAL LANTERN)
...,...,...
1732,0.010335,"(LUNCH BAG SPACEBOY DESIGN, LUNCH BAG VINTAGE ..."
1733,0.011257,"(LUNCH BAG VINTAGE DOILY, LUNCH BAG APPLE DESIGN)"
1734,0.011451,"(LOVE HOT WATER BOTTLE, HOT WATER BOTTLE KEEP ..."
1735,0.013052,"(HOT WATER BOTTLE KEEP CALM, CHOCOLATE HOT WAT..."


In [23]:
#generating rules
rules = association_rules(itemsets, metric="lift", min_threshold=1)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(HAND WARMER UNION JACK),(HAND WARMER OWL DESIGN),0.023241,0.031732,0.011451,0.492693,15.526613,0.010713,1.908643
1,(HAND WARMER OWL DESIGN),(HAND WARMER UNION JACK),0.031732,0.023241,0.011451,0.360856,15.526613,0.010713,1.528230
2,(HAND WARMER UNION JACK),(HAND WARMER SCOTTY DOG DESIGN),0.023241,0.026589,0.010432,0.448852,16.881086,0.009814,1.766151
3,(HAND WARMER SCOTTY DOG DESIGN),(HAND WARMER UNION JACK),0.026589,0.023241,0.010432,0.392336,16.881086,0.009814,1.607399
4,(WHITE HANGING HEART T-LIGHT HOLDER),(ASSORTED COLOUR BIRD ORNAMENT),0.109656,0.070597,0.015672,0.142920,2.024459,0.007931,1.084384
...,...,...,...,...,...,...,...,...,...
2809,(HOT WATER BOTTLE KEEP CALM),(LOVE HOT WATER BOTTLE),0.037943,0.023096,0.011451,0.301790,13.067012,0.010574,1.399156
2810,(HOT WATER BOTTLE KEEP CALM),(CHOCOLATE HOT WATER BOTTLE),0.037943,0.041630,0.013052,0.343990,8.262971,0.011472,1.460907
2811,(CHOCOLATE HOT WATER BOTTLE),(HOT WATER BOTTLE KEEP CALM),0.041630,0.037943,0.013052,0.313520,8.262971,0.011472,1.401435
2812,(HAND WARMER RED LOVE HEART),(HAND WARMER OWL DESIGN),0.019262,0.031732,0.010577,0.549118,17.304786,0.009966,2.147499


In [48]:
#Filtering rules based on condition
rules[ (rules['lift'] >= 1) &
       (rules['confidence'] >= 0.7) ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
16,(POPPY'S PLAYHOUSE KITCHEN),(POPPY'S PLAYHOUSE BEDROOM),0.021155,0.020524,0.015187,0.717890,34.978040,0.014753,3.471964
17,(POPPY'S PLAYHOUSE BEDROOM),(POPPY'S PLAYHOUSE KITCHEN),0.020524,0.021155,0.015187,0.739953,34.978040,0.014753,3.764105
74,"(ALARM CLOCK BAKELIKE PINK, ALARM CLOCK BAKELI...",(ALARM CLOCK BAKELIKE RED),0.020281,0.050995,0.015187,0.748804,14.683965,0.014153,3.777945
154,"(JUMBO BAG PINK POLKADOT, LUNCH BAG RED RETROS...",(JUMBO BAG RED RETROSPOT),0.015672,0.101504,0.012082,0.770898,7.594744,0.010491,3.921813
168,"(JUMBO BAG PINK POLKADOT, JUMBO STORAGE BAG SUKI)",(JUMBO BAG RED RETROSPOT),0.024988,0.101504,0.020039,0.801942,7.900583,0.017502,4.536523
...,...,...,...,...,...,...,...,...,...
2736,(REGENCY TEA PLATE PINK),"(REGENCY TEA PLATE ROSES, REGENCY TEA PLATE GR...",0.014799,0.015478,0.012373,0.836066,54.016650,0.012144,6.005585
2743,(GARDENERS KNEELING PAD CUP OF TEA),(GARDENERS KNEELING PAD KEEP CALM),0.036778,0.044299,0.026492,0.720317,16.260379,0.024863,3.417082
2765,(SET OF 3 WOODEN TREE DECORATIONS),(SET OF 3 WOODEN STOCKING DECORATION),0.014216,0.015478,0.010626,0.747440,48.290734,0.010406,3.898175
2767,(SET OF 3 WOODEN TREE DECORATIONS),(SET OF 3 WOODEN HEART DECORATIONS),0.014216,0.021591,0.010238,0.720137,33.352840,0.009931,3.496021


In [55]:
#finding the recommended itemset based on the item already bought
output= rules.loc[rules['antecedents'] == rules['antecedents'][1]]
output[['consequents']]

,consequents
1,(HAND WARMER UNION JACK)
692,(HAND WARMER SCOTTY DOG DESIGN)
696,(HAND WARMER RED RETROSPOT)
711,(HAND WARMER BIRD DESIGN)
2813,(HAND WARMER RED LOVE HEART)
